In [1]:
import pandas as pd
import numpy as np
import logging
import warnings
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.io.json import json_normalize

#np.set_printoptions(precision=2, suppress=True, linewidth=120)



In [2]:
#!jupyter labextension install @ryantam626/jupyterlab_code_formatter
#!pip install jupyterlab_code_formatter
#!jupyter serverextension enable --py jupyterlab_code_formatter

In [3]:
#!pip install Autopep8

In [4]:
#!pip install cufflinks plotly
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [5]:
# read input files
# import fastparquet

time_column_name = 'Snapshot_Date_Short'
sort_cols = ['Snapshot_Date_Short','End_of_Month']
output_data_path = './output-data/'
final_merge_df = pd.read_csv(output_data_path + 'final_merge_df.csv')
final_merge_df.sort_values(by=sort_cols,inplace=True)

In [6]:
# Calculate variance
final_merge_df.info(verbose=True)
display(final_merge_df.tail(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 479
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Snapshot_Date_Short             480 non-null    object 
 1   Relative_Offset                 480 non-null    int64  
 2   origin                          480 non-null    object 
 3   predicted                       480 non-null    int64  
 4   horizon_origin                  480 non-null    int64  
 5   Relative_Snapshot_Month_Offset  480 non-null    int64  
 6   End_of_Month                    480 non-null    object 
 7   Relative_Month_Offset           480 non-null    int64  
 8   Pipeline                        480 non-null    int64  
 9   Pipeline_3_Month_Rolling_Avg    480 non-null    int64  
 10  Pipeline_3M_Forward_Avg         480 non-null    int64  
 11  Pipeline_6M_Forward_Avg         480 non-null    int64  
 12  Pipeline_6M_Rolling_Avg         480 

,Snapshot_Date_Short,Relative_Offset,origin,predicted,horizon_origin,Relative_Snapshot_Month_Offset,End_of_Month,Relative_Month_Offset,Pipeline,Pipeline_3_Month_Rolling_Avg,...,Current_Opp._Period_Value,Conversions,Pipeline_Trend,Pipeline_Trend_Avg,Billable_Headcount,sp500,Revenue,Revenue_Forecast,Diff,Diff%
470,2021-02-28,-6,2020-10-31,12990285,4,0,2020-08-31,-6,12986821,13771950,...,40197662,10601989,34129552,11376517,576,3500.31,12986818,12990285,-3467,-0.0267
471,2021-02-28,-5,2020-10-31,11708012,4,0,2020-09-30,-5,11695096,13225797,...,33724833,10437439,32049434,10683144,584,3363.00,11695095,11708012,-12917,-0.1103
472,2021-02-28,-4,2020-10-31,15098325,4,0,2020-10-31,-4,15433515,13371811,...,49246225,14536849,35050408,11683469,579,3269.96,15433514,15098325,335189,2.2200
473,2021-02-28,-3,2020-10-31,13665978,4,0,2020-11-30,-3,14031333,13719981,...,34674593,9928454,35824284,11941428,576,3621.63,14031331,13665978,365353,2.6734
474,2021-02-28,-2,2020-10-31,14047757,4,0,2020-12-31,-2,14248282,14571044,...,26494658,17927536,38679414,12893138,530,3756.07,14248280,14047757,200523,1.4274
475,2021-02-28,-1,2020-10-31,14043333,4,0,2021-01-31,-1,13738530,14006048,...,24310652,14446751,0,0,530,3714.24,13738529,14043333,-304804,-2.1705
476,2021-02-28,0,2020-10-31,12882353,4,0,2021-02-28,0,12974908,13653907,...,1080000,222513,39878266,13292755,526,3830.17,1285947,12882353,-11596406,-90.0178
477,2021-02-28,1,2020-10-31,14248092,4,0,2021-03-31,1,17476023,14729820,...,0,0,0,0,525,0.00,1055739,14248092,-13192353,-92.5903
478,2021-02-28,2,2020-10-31,12646611,4,0,2021-04-30,2,11911579,14120837,...,0,0,0,0,522,0.00,0,12646611,-12646611,-100.0000
479,2021-02-28,3,2020-10-31,12942460,4,0,2021-05-31,3,7608693,12332099,...,0,0,0,0,522,0.00,0,12942460,-12942460,-100.0000


In [20]:
# pivot by EOM1, Fin_Entity_ID
pd.set_option('display.float_format', lambda x: '%.1f' % x) #


cols = ['Pipeline','Revenue','predicted','Diff','Diff%']
final_merge_pivot1 = final_merge_df.query('Relative_Offset >= -3 and Relative_Snapshot_Month_Offset >= -3').pivot_table(index=['Snapshot_Date_Short','End_of_Month'], #columns='Relative_Offset',
        values=cols, aggfunc={'predicted':np.sum,'Pipeline':np.sum,'Revenue':np.sum,'Diff':np.sum, 'Diff%':np.mean}, margins=None)
#final_merge_pivot1 = \
#    final_merge_pivot1['End_of_Month'] = pd.to_datetime(final_merge_pivot1['End_of_Month']).dt.date
# final_merge_pivot1.reset_index(level=final_merge_pivot1.index.names)

# show_stats(revtime_pivot1)
final_merge_pivot1.sort_values(by=sort_cols,inplace=True)
#reorder columns
cols_order = [2,3,4,0,1]
final_merge_pivot1 = final_merge_pivot1[[final_merge_pivot1.columns[i] for i in cols_order]]

# set ALL float columns to '${:,.2f}' formatting (including the percentage)
#format_dict = {col_name: '${:,.1f}' for col_name in final_merge_pivot1.select_dtypes(float).columns}
# override the percentage column
format_dict = {col_name: '{:,}' for col_name in final_merge_pivot1.columns}
format_dict['Diff%'] = '{:.2f}'


# Format with commas and round off to two decimal places in pandas 
# final_merge_pivot1 = final_merge_pivot1[[['Snapshot_Date_Short','End_of_Month']+cols]]
final_merge_pivot1.tail(28).style.format(format_dict)#.style.format('{:,}')# .query('(End_of_Month == @current_eom)')

In [8]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
import plotly.graph_objs as go
fig_width = 1500
fig_height = 700
df = final_merge_df.query('Relative_Snapshot_Month_Offset == 0 and Relative_Offset >= 0 and Relative_Offset <= 3')

df['End_of_Month'] = pd.to_datetime(df['End_of_Month'
                                    ]).dt.date.astype(str)
df['Snapshot_Date_Short'] = pd.to_datetime(df['Snapshot_Date_Short'
        ]).dt.date.astype(str)

x = 'End_of_Month'
y = 'Pipeline'
y1 = 'Revenue'
y2 = 'predicted'
y3 = 'Diff'

trace1 = go.Bar(x=df[x], y=df[y], text=df[y], name=y,
                marker_color='green')
trace2 = go.Bar(x=df[x], y=df[y1], text=df[y1], name=y1,
                marker_color='grey')
trace3 = go.Bar(x=df[x], y=df[y2], text=df[y2], name=y2,
                marker_color='blue')  # mode="lines",

data = [trace1, trace2, trace3]

layout = go.Layout(
    title_text='Revenue Forecast Snapshot Date' + ' '
        + df['Snapshot_Date_Short'].max(),
    barmode='group',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    width=fig_width,
    height=fig_height,
    )
fig = go.Figure(data=data, layout=layout)
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  #textfont_color='white'
                  )

trace4 = go.Scatter(x=df[x], y=df[y3], text=df[y3], name=y3,
                marker_color='red', mode="markers+lines")
fig.add_trace(trace4)   
          
'''
fig.add_trace(go.Scatter(
    x=df[x],
    y=df[y3],
    text=df[y3],
    mode='lines',
    line=go.scatter.Line(color='red'),
    name='Diff'
    ))
'''
fig.update_xaxes(tickangle=90, title_text=x)

        # title_font = {"size": 20},
        # title_standoff = 25

iplot(fig)


In [9]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
import plotly.graph_objs as go
fig_width = 1500
fig_height = 700
df = final_merge_df.query('Relative_Snapshot_Month_Offset == -1 and Relative_Offset >= 0 and Relative_Offset <= 3')

df['End_of_Month'] = pd.to_datetime(df['End_of_Month'
                                    ]).dt.date.astype(str)
df['Snapshot_Date_Short'] = pd.to_datetime(df['Snapshot_Date_Short'
        ]).dt.date.astype(str)

x = 'End_of_Month'
y = 'Pipeline'
y1 = 'Revenue'
y2 = 'predicted'
y3 = 'Diff'

trace1 = go.Bar(x=df[x], y=df[y], text=df[y], name=y,
                marker_color='green')
trace2 = go.Bar(x=df[x], y=df[y1], text=df[y1], name=y1,
                marker_color='grey')
trace3 = go.Bar(x=df[x], y=df[y2], text=df[y2], name=y2,
                marker_color='blue')  # mode="lines",

data = [trace1, trace2, trace3]

layout = go.Layout(
    title_text='Revenue Forecast Snapshot Date' + ' '
        + df['Snapshot_Date_Short'].max(),
    barmode='group',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    width=fig_width,
    height=fig_height,
    )
fig = go.Figure(data=data, layout=layout)
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  #textfont_color='white'
                  )

trace4 = go.Scatter(x=df[x], y=df[y3], text=df[y3], name=y3,
                marker_color='red', mode="markers+lines")
fig.add_trace(trace4)   
          
'''
fig.add_trace(go.Scatter(
    x=df[x],
    y=df[y3],
    text=df[y3],
    mode='lines',
    line=go.scatter.Line(color='red'),
    name='Diff'
    ))
'''
fig.update_xaxes(tickangle=90, title_text=x)

        # title_font = {"size": 20},
        # title_standoff = 25

iplot(fig)


In [10]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
import plotly.graph_objs as go
fig_width = 1500
fig_height = 700
df = final_merge_df.query('Relative_Snapshot_Month_Offset == -2 and Relative_Offset >= 0 and Relative_Offset <= 3')

df['End_of_Month'] = pd.to_datetime(df['End_of_Month'
                                    ]).dt.date.astype(str)
df['Snapshot_Date_Short'] = pd.to_datetime(df['Snapshot_Date_Short'
        ]).dt.date.astype(str)

x = 'End_of_Month'
y = 'Pipeline'
y1 = 'Revenue'
y2 = 'predicted'
y3 = 'Diff'

trace1 = go.Bar(x=df[x], y=df[y], text=df[y], name=y,
                marker_color='green')
trace2 = go.Bar(x=df[x], y=df[y1], text=df[y1], name=y1,
                marker_color='grey')
trace3 = go.Bar(x=df[x], y=df[y2], text=df[y2], name=y2,
                marker_color='blue')  # mode="lines",

data = [trace1, trace2, trace3]

layout = go.Layout(
    title_text='Revenue Forecast Snapshot Date' + ' '
        + df['Snapshot_Date_Short'].max(),
    barmode='group',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    width=fig_width,
    height=fig_height,
    )
fig = go.Figure(data=data, layout=layout)
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  #textfont_color='white'
                  )

trace4 = go.Scatter(x=df[x], y=df[y3], text=df[y3], name=y3,
                marker_color='red', mode="markers+lines")
fig.add_trace(trace4)   
          
'''
fig.add_trace(go.Scatter(
    x=df[x],
    y=df[y3],
    text=df[y3],
    mode='lines',
    line=go.scatter.Line(color='red'),
    name='Diff'
    ))
'''
fig.update_xaxes(tickangle=90, title_text=x)

        # title_font = {"size": 20},
        # title_standoff = 25

iplot(fig)


In [11]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
import plotly.graph_objs as go
fig_width = 1500
fig_height = 700
df = final_merge_df.query('Relative_Snapshot_Month_Offset == 0 and Relative_Offset >= -11 and Relative_Offset <= 3')

df['End_of_Month'] = pd.to_datetime(df['End_of_Month'
                                    ]).dt.date.astype(str)
df['Snapshot_Date_Short'] = pd.to_datetime(df['Snapshot_Date_Short'
        ]).dt.date.astype(str)

x = 'End_of_Month'
y = 'Pipeline'
y1 = 'Revenue'
y2 = 'predicted'
y3 = 'Diff'
y4 = 'Diff%'

trace1 = go.Bar(x=df[x], y=df[y3], text=df[y3], name=y3, 
                # marker_color='red'
                )
#fig.add_trace(trace1)
data = [trace1]

layout = go.Layout(
    title_text='Revenue Forecast Diff Snapshot Date' + ' '
        + df['Snapshot_Date_Short'].max(),
    barmode='group',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    width=fig_width,
    height=fig_height
    )
fig = go.Figure(data=data, layout=layout)
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  #textfont_color='white'
                  )

#trace4 = go.Scatter(x=df[x], y=df[y3], text=df[y3], name=y3,
#                marker_color='red', mode="markers+lines")
#fig.add_trace(trace4)   
          
'''
fig.add_trace(go.Scatter(
    x=df[x],
    y=df[y3],
    text=df[y3],
    mode='lines',
    line=go.scatter.Line(color='red'),
    name='Diff'
    ))
'''
fig.update_xaxes(tickangle=90, title_text=x)

        # title_font = {"size": 20},
        # title_standoff = 25

iplot(fig)


In [12]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
import plotly.graph_objs as go
fig_width = 1500
fig_height = 700
df = final_merge_df.query('Relative_Snapshot_Month_Offset == 0 and Relative_Offset >= -11 and Relative_Offset <= 3')

df['End_of_Month'] = pd.to_datetime(df['End_of_Month'
                                    ]).dt.date.astype(str)
df['Snapshot_Date_Short'] = pd.to_datetime(df['Snapshot_Date_Short'
        ]).dt.date.astype(str)

x = 'End_of_Month'
y = 'Pipeline'
y1 = 'Revenue'
y2 = 'predicted'
y3 = 'Diff'
y4 = 'Diff%'

trace1 = go.Bar(x=df[x], y=df[y4], text=df[y4], name=y4, 
                # marker_color='green'
                )
#fig.add_trace(trace1)
data = [trace1]

layout = go.Layout(
    title_text='Revenue Forecast Diff% Snapshot Date' + ' '
        + df['Snapshot_Date_Short'].max(),
    barmode='group',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    width=fig_width,
    height=fig_height
    )
fig = go.Figure(data=data, layout=layout)
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  #textfont_color='auto'
                  )

#trace4 = go.Scatter(x=df[x], y=df[y3], text=df[y3], name=y3,
#                marker_color='red', mode="markers+lines")
#fig.add_trace(trace4)   
          
'''
fig.add_trace(go.Scatter(
    x=df[x],
    y=df[y3],
    text=df[y3],
    mode='lines',
    line=go.scatter.Line(color='red'),
    name='Diff'
    ))
'''
fig.update_xaxes(tickangle=90, title_text=x)

        # title_font = {"size": 20},
        # title_standoff = 25

iplot(fig)


In [13]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
import plotly.graph_objs as go
df = final_merge_df.query('Relative_Snapshot_Month_Offset == 0 and Relative_Snapshot_Month_Offset >= -3 and Relative_Offset >= -3')
'''
df_sum = final_merge_df.pivot_table(index=['Snapshot_Date_Short',
                                   'Relative_Snapshot_Month_Offset',
                                   'End_of_Month', 'Relative_Offset'],
                                   values=['predicted', 'Revenue',
                                   'Pipeline'], aggfunc=np.sum,
                                   margins=False)  # columns='Relative_Offet',
df_sum = df_sum.reset_index(level=df_sum.index.names)
'''
df['End_of_Month'] = pd.to_datetime(df['End_of_Month'
                                    ]).dt.date.astype(str)
df['Snapshot_Date_Short'] = pd.to_datetime(df['Snapshot_Date_Short'
        ]).dt.date.astype(str)

x = 'End_of_Month'
y = 'predicted'
y1 = 'Pipeline'
y2 = 'Revenue'
fig = px.bar(  # color=y,
               # facet_row='Snapshot_Date_Short',
    df,
    x=x,
    y=y,
    text=y,
    barmode='group',
    facet_col='Snapshot_Date_Short',
    width=fig_width,
    height=fig_height,
    title='Revenue Forecast',
    )
trace2 = go.Bar(x=df[x], y=df[y1], text=df[y1], name=y1,
                marker_color='grey')
trace2.update(legendgroup='trendline', showlegend=False)
fig.add_trace(trace2, row='all', col='all', exclude_empty_subplots=True)
fig.update_traces(selector=-1, showlegend=True)
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  textfont_color='black')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

'''
fig.add_trace(go.Scatter(
    x=df_sum[x],
    y=df_sum[y2],
    text=df_sum[y2],
    mode='lines',
    line=go.scatter.Line(color='green'),
    name='Pipeline',
    ))
'''
fig.update_xaxes(tickangle=90, title_text=x)

        # title_font = {"size": 20},
        # title_standoff = 25

fig.show()


In [14]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
import plotly.graph_objs as go
fig_width = 1600
fig_height = 600
df = final_merge_df.query('Relative_Snapshot_Month_Offset >= -3')
df1 = df.query('Relative_Snapshot_Month_Offset == 0')
df2 = df.query('Relative_Snapshot_Month_Offset == -1')
df3 = df.query('Relative_Snapshot_Month_Offset == -2')
df4 = df.query('Relative_Snapshot_Month_Offset == -3')
'''
df_sum = final_merge_df.pivot_table(index=['Snapshot_Date_Short',
                                   'Relative_Snapshot_Month_Offset',
                                   'End_of_Month', 'Relative_Offset'],
                                   values=['predicted', 'Revenue',
                                   'Pipeline'], aggfunc=np.sum,
                                   margins=False)  # columns='Relative_Offet',
df_sum = df_sum.reset_index(level=df_sum.index.names)
'''

x = 'End_of_Month'
y = 'predicted'
y1 = 'Pipeline'
y2 = 'Revenue'
fig = px.bar(  # color=y1,
               # facet_row='Snapshot_Date_Short',
    df,
    x=x,
    y=y,
    text=y,
    barmode='group',
    facet_col='Snapshot_Date_Short',
    width=fig_width,
    height=fig_height,
    title='Revenue Forecast',
    )
trace1 = go.Bar(x=df1[x], y=df1[y1], text=df1[y1], name=y1,
                marker_color='grey')
trace2 = go.Bar(x=df2[x], y=df2[y1], text=df2[y1], name=y1,
                marker_color='grey')
trace3 = go.Bar(x=df3[x], y=df3[y1], text=df3[y1], name=y1,
                marker_color='grey')
trace4 = go.Bar(x=df4[x], y=df4[y1], text=df4[y1], name=y1,
                marker_color='grey')
#trace.update(showlegend=False)
trace1.update(showlegend=False)
trace2.update(showlegend=False)
trace3.update(showlegend=False)
trace4.update(showlegend=False)
fig.add_trace(trace1, row=1, col=1, exclude_empty_subplots=True)
fig.add_trace(trace2, row=1, col=2, exclude_empty_subplots=True)
fig.add_trace(trace3, row=1, col=3, exclude_empty_subplots=True)
fig.add_trace(trace4, row=1, col=4, exclude_empty_subplots=True)
fig.update_traces(selector=-1, showlegend=True)
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  textfont_color='black')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

'''
fig.add_trace(go.Scatter(
    x=df_sum[x],
    y=df_sum[y2],
    text=df_sum[y2],
    mode='lines',
    line=go.scatter.Line(color='green'),
    name='Pipeline',
    ))
'''

fig.update_xaxes(tickangle=90, title_text=x)

        # title_font = {"size": 20},
        # title_standoff = 25

fig.show()


In [ ]:
display(df.info(verbose=True))
df.tail(20)

In [127]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
import plotly.graph_objs as go
fig_width = 1600
fig_height = 600
df = final_merge_df.query('Relative_Snapshot_Month_Offset >= -3')

# df_sum = final_merge_df.pivot_table(index=['Snapshot_Date_Short',
#                                    'Relative_Snapshot_Month_Offset',
#                                    'End_of_Month', 'Relative_Offset'],
#                                    values=['predicted', 'Revenue',
#                                    'Pipeline'], aggfunc=np.sum,
#                                    margins=False)  # columns='Relative_Offet',
# df_sum = df_sum.reset_index(level=df_sum.index.names)

df['End_of_Month'] = pd.to_datetime(df['End_of_Month'
                                    ]).dt.date.astype(str)
df['Snapshot_Date_Short'] = pd.to_datetime(df['Snapshot_Date_Short'
        ]).dt.date.astype(str)

x = 'End_of_Month'
y = 'Pipeline'
y1 = 'Revenue'
y2 = 'predicted'
y3 = 'Diff'
y4 = 'Diff%'

fig = px.bar( 
    df,
    x=x,
    y=y2,
    text=y2,
    barmode='group',
#    facet_row='Snapshot_Date_Short',
    facet_col='Snapshot_Date_Short',
    facet_col_wrap=1,
    width=fig_width,
    height=fig_height,
    title='Revenue Forecast'
    )
trace1 = go.Bar(x=df[x], y=df[y1], text=df[y1], marker_color='grey', offset = 1)  # name=y1,
#trace2 = go.Bar(x=df[x], y=df[y], text=df[y], marker_color='green', offset = 2)  # name=y1,
fig.add_trace(trace1, row='all', col='all', exclude_empty_subplots=True)
#fig.add_trace(trace2, row='all', col='all', exclude_empty_subplots=True)
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  #textfont_color='black'
                  )
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

'''
fig.add_trace(go.Scatter(
    x=df_sum[x],
    y=df_sum[y2],
    text=df_sum[y2],
    mode='lines',
    line=go.scatter.Line(color='green'),
    name='Pipeline',
    ))
'''

fig.update_xaxes(tickangle=90, title_text=x)

        # title_font = {"size": 20},
        # title_standoff = 25

fig.show()


ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'offset'

Did you mean "fill"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            customdata .
        dx
            Sets the x coordinate ste

In [73]:
df_grp = final_merge_df.groupby(by=['Snapshot_Date_Short',
                                'End_of_Month', 'Relative_Offset']).sum()
df_grp = df_grp.reset_index(level=df_grp.index.names)
df_grp

,Snapshot_Date_Short,End_of_Month,Relative_Offset,predicted,horizon_origin,Relative_Snapshot_Month_Offset,Relative_Month_Offset,Pipeline,Pipeline_3_Month_Rolling_Avg,Pipeline_3M_Forward_Avg,...,Current_Opp._Period_Value,Conversions,Pipeline_Trend,Pipeline_Trend_Avg,Total_Headcount,sp500,Revenue,Revenue_Forecast,Diff,Diff%
0,2018-07-31,2017-08-31,-11,0,0,-30,-41,13644814,12932411,13533999,...,0,6762094,0,0,519,2471.7,13644814,13644814,13644814,inf
1,2018-07-31,2017-09-30,-10,0,0,-30,-40,13257051,12904745,13462141,...,0,4530171,0,0,519,2519.4,13257051,13257051,13257051,inf
2,2018-07-31,2017-10-31,-9,0,0,-30,-39,13700132,13533999,12981146,...,0,6782319,0,0,539,2575.3,13700132,13700132,13700132,inf
3,2018-07-31,2017-11-30,-8,0,0,-30,-38,13429239,13462141,13310566,...,0,7408248,0,0,548,2647.6,13429239,13429239,13429239,inf
4,2018-07-31,2017-12-31,-7,0,0,-30,-37,11814066,12981146,13151731,...,0,4170817,0,0,531,2673.6,11814066,11814066,11814066,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,2021-01-31,2020-12-31,-1,14179135,4,0,-1,14243578,14569476,14606262,...,25759134,17927536,38679414,12893138,529,3756.1,14243576,14179135,64441,0.5
461,2021-01-31,2021-01-31,0,13477000,4,0,0,14400426,14225113,15124786,...,5585887,8068099,42197970,14065990,525,3801.2,1443196,13477000,-12033804,-89.3
462,2021-01-31,2021-02-28,1,13880067,4,0,1,15174782,14606262,14092614,...,0,0,0,0,525,0.0,1296579,13880067,-12583488,-90.7
463,2021-01-31,2021-03-31,2,14073999,4,0,2,15799150,15124786,11554489,...,0,0,0,0,524,0.0,1066371,14073999,-13007628,-92.4


In [74]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import plotly.express as px
fig_width = 1600
fig_height = 800
df = df
x = 'End_of_Month'
y = 'predicted'
y1 = 'Pipeline'
y2 = 'Revenue'
color = 'Snapshot_Date_Short'
fig = px.bar(
    df,
    x=x,
    y=y,
    color=color,
    text=y,
    width=fig_width,
    height=fig_height,
    title='Revenue Forecast',
    )
fig.update_traces(texttemplate='%{text:.3s}', textposition='auto',
                  textfont_color='white')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


In [75]:
fig2 = px.bar(
        df1, 
        x=x, 
        y=y1,
        color=color,
        text=y1,
        width=fig_width,
        height=fig_height,
        title="Pipeline"
    )
fig2.update_traces(texttemplate='%{text:.3s}', textposition='auto', textfont_color="white")
fig2.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig2.show()

In [76]:
fig3 = px.bar(
        df2, 
        x=x, 
        y=y2,
        color=color,
        text=y2,
        width=fig_width,
        height=fig_height,
        title="Pipeline_3M_Forward_Avg"
    )
fig3.update_traces(texttemplate='%{text:.3s}', textposition='auto', textfont_color="white")
fig3.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig3.show()